# 0. ETF Selection

We select the SPDR Gold Shares (GLD) ETF as the gold ETF. It is traded on Nasdaq, the currency is USD.

Similarly, we choose the Amundi CAC 40 UCITS ETF-C (C40.PA) as the equity ETF. It will track the CAC 40 index of France. It is traded on Paris Euronext, the currency is EUR.

The currency for Bitcoin is USD.

Data source: https://finance.yahoo.com/

# 1. Data Importing

In [1]:
import arch
import holidays
import pmdarima
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.stattools import adfuller, coint
from statsmodels.tsa.vector_ar.vecm import VECM
from sklearn import preprocessing
%matplotlib inline

In [2]:
gold_df = pd.read_csv("data/SPDR Gold Shares (GLD) Jan - Dec 2020.csv")
equity_df = pd.read_csv("data/Amundi CAC 40 UCITS ETF-C (C40.PA) Jan 2020 - Dec 2020.csv")
bitcoin_df = pd.read_csv('data/Bitcoin USD (BTC-USD) Jan 2020 - Dec 2020.csv')

Convert the data into the datetime format and make it the index to query the dataframe easier.

In [3]:
def convert_df(df):
    df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")
    df.set_index("Date", inplace=True)   
    return df

gold_df = convert_df(gold_df)
equity_df = convert_df(equity_df)
bitcoin_df = convert_df(bitcoin_df)


We use the common subset of days for all 3 time series to make them comparable and to run the test for cointegration in question 7.

In [4]:
missing = set(equity_df.index) - set(gold_df.index)

In [5]:
def remove_missing_days(df, indices, missing):
    return df.loc[[index for index in indices if index not in missing]]

In [6]:
gold_df = remove_missing_days(gold_df, equity_df.index, missing)
equity_df = remove_missing_days(equity_df, equity_df.index, missing)
bitcoin_df = remove_missing_days(bitcoin_df, equity_df.index, missing)

In [7]:
assert equity_df.shape == gold_df.shape
assert equity_df.shape == bitcoin_df.shape

We then fill some missing values in the equity series with its old values to avoid data leakage.

In [8]:
equity_df = equity_df.ffill()

Verify that the time range is correct.

In [9]:
gold_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,143.860001,144.210007,143.399994,143.949997,143.949997,7733800
2020-01-03,145.750000,146.320007,145.399994,145.860001,145.860001,12272800
2020-01-06,148.440002,148.479996,146.949997,147.389999,147.389999,14403300
2020-01-07,147.570007,148.139999,147.429993,147.970001,147.970001,7978500
2020-01-08,148.490005,148.610001,146.139999,146.860001,146.860001,22248500


In [10]:
gold_df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-12-23,175.100006,176.210007,175.059998,175.649994,175.649994,6542800
2020-12-24,175.550003,176.369995,175.509995,176.350006,176.350006,3695400
2020-12-28,177.259995,177.910004,175.630005,175.710007,175.710007,7778700
2020-12-29,176.250000,176.970001,175.570007,176.350006,176.350006,5983700
2020-12-30,176.440002,177.720001,176.440002,177.699997,177.699997,5914000


In [11]:
equity_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,88.589996,89.239998,88.589996,89.239998,89.239998,124.0
2020-01-03,88.489998,88.889999,88.309998,88.769997,88.769997,0.0
2020-01-06,88.529999,88.599998,87.750000,88.559998,88.559998,563.0
2020-01-07,88.839996,89.260002,88.500000,88.500000,88.500000,0.0
2020-01-08,88.120003,89.300003,88.120003,89.139999,89.139999,212.0


In [12]:
equity_df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-12-23,82.480003,83.330002,82.000000,83.250000,83.250000,68117.0
2020-12-24,83.419998,83.550003,82.930000,83.139999,83.139999,32892.0
2020-12-28,83.849998,84.290001,83.410004,84.160004,84.160004,20640.0
2020-12-29,84.440002,84.680000,84.320000,84.449997,84.449997,20668.0
2020-12-30,84.489998,84.699997,84.300003,84.339996,84.339996,3829.0


In [13]:
bitcoin_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,6985.470215,20802083465
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,7344.884277,28111481031
2020-01-06,7410.452148,7781.867188,7409.292969,7769.219238,7769.219238,23276261598
2020-01-07,7768.682129,8178.215820,7768.227539,8163.692383,8163.692383,28767291326
2020-01-08,8161.935547,8396.738281,7956.774414,8079.862793,8079.862793,31672559264


In [14]:
bitcoin_df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-12-23,23781.974609,24024.490234,22802.646484,23241.345703,23241.345703,51146161904
2020-12-24,23240.203125,23768.337891,22777.597656,23735.949219,23735.949219,41080759712
2020-12-28,26280.822266,27389.111328,26207.640625,27084.808594,27084.808594,49056742892
2020-12-29,27081.810547,27370.720703,25987.298828,27362.437500,27362.437500,45265946774
2020-12-30,27360.089844,28937.740234,27360.089844,28840.953125,28840.953125,51287442703


# 2. Data Processing

We use adjusted close prices to calculate the daily returns. Adjusted close prices are the prices that already take into account stock split and dividends, which reflex more accurate the change of the prices.

In [15]:
equity_df["Daily Return"] = equity_df["Adj Close"].pct_change(1)
equity_df.head()

,Open,High,Low,Close,Adj Close,Volume,Daily Return
Date,,,,,,,
2020-01-02,88.589996,89.239998,88.589996,89.239998,89.239998,124.0,NaN
2020-01-03,88.489998,88.889999,88.309998,88.769997,88.769997,0.0,-0.005267
2020-01-06,88.529999,88.599998,87.750000,88.559998,88.559998,563.0,-0.002366
2020-01-07,88.839996,89.260002,88.500000,88.500000,88.500000,0.0,-0.000677
2020-01-08,88.120003,89.300003,88.120003,89.139999,89.139999,212.0,0.007232


In [16]:
gold_df["Daily Return"] = gold_df["Adj Close"].pct_change(1)
gold_df.head()

,Open,High,Low,Close,Adj Close,Volume,Daily Return
Date,,,,,,,
2020-01-02,143.860001,144.210007,143.399994,143.949997,143.949997,7733800,NaN
2020-01-03,145.750000,146.320007,145.399994,145.860001,145.860001,12272800,0.013269
2020-01-06,148.440002,148.479996,146.949997,147.389999,147.389999,14403300,0.010489
2020-01-07,147.570007,148.139999,147.429993,147.970001,147.970001,7978500,0.003935
2020-01-08,148.490005,148.610001,146.139999,146.860001,146.860001,22248500,-0.007502


In [17]:
bitcoin_df["Daily Return"] = bitcoin_df["Adj Close"].pct_change(1)
bitcoin_df.head()

,Open,High,Low,Close,Adj Close,Volume,Daily Return
Date,,,,,,,
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,6985.470215,20802083465,NaN
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,7344.884277,28111481031,0.051452
2020-01-06,7410.452148,7781.867188,7409.292969,7769.219238,7769.219238,23276261598,0.057773
2020-01-07,7768.682129,8178.215820,7768.227539,8163.692383,8163.692383,28767291326,0.050774
2020-01-08,8161.935547,8396.738281,7956.774414,8079.862793,8079.862793,31672559264,-0.010269


# 5. Category 1 Models: Just use 1 variable.

In [18]:
# Helper
df_names = {0: "gold ETF", 1: "equity ETF", 2: "Bitcoin"}
dfs = [gold_df, equity_df, bitcoin_df]
def get_data(df, month_start, month_end, column=None):
    data = df[(df.index >= f"2020-{month_start:02d}-01") & (df.index < f"2020-{month_end:02d}-01")]
    if column: 
        data = data[column]
    return data

In [19]:
def summarize_data(df):
    data = get_data(df, 3, 12, "Adj Close")
    moving_avg = data.rolling(20, min_periods=1).mean()
    return get_data(moving_avg, 4, 12)


In [20]:
def get_data(df, month_start, month_end, column):
    return df[(df.index >= f"2020-{month_start:02d}-01") & (df.index < f"2020-{month_end:02d}-01")][column]

In [21]:
def fit_arima(data, exog= None):
    model = ARIMA(data, exog=exog, order=(2,0,2))
    model_fit = model.fit()
    return model_fit

In [22]:
def fit_garch(data, garch_type="GARCH"):
    if garch_type == "TARCH":
        garch = arch.arch_model(data, vol='TGARCH', p=1, o=1, q=1, power=1)
    else:
        garch = arch.arch_model(data, vol=garch_type, p=1, o=0, q=1)
    garch_fitted = garch.fit()
    print(garch_fitted.summary())
    if garch_type != "FIGARCH":
        omega = garch_fitted.params["omega"]
        alpha = garch_fitted.params["alpha[1]"]
        beta = garch_fitted.params["beta[1]"]
        print(f"Unconditional variance: {omega/(1 - alpha - beta)}")
    return garch_fitted

In [23]:
bitcoin_q2 = get_data(bitcoin_df, 4, 6, "Daily Return")
bitcoin_q3 = get_data(bitcoin_df, 7, 9, "Daily Return")
bitcoin_q4 = get_data(bitcoin_df, 10, 12, "Daily Return")

In [24]:
equity_q2 = get_data(equity_df, 4, 6, "Daily Return")
equity_q3 = get_data(equity_df, 7, 9, "Daily Return")
equity_q4 = get_data(equity_df, 10, 12, "Daily Return")

In [25]:
gold_q2 = get_data(gold_df, 4, 6, "Daily Return")
gold_q3 = get_data(gold_df, 7, 9, "Daily Return")
gold_q4 = get_data(gold_df, 10, 12, "Daily Return")

In [26]:
bitcoin_q2.shape

(39,)

In [27]:
bitcoin_q3.shape

(43,)

In [28]:
bitcoin_q4.shape

(42,)

In [29]:
gold_q2.shape

(39,)

In [30]:
gold_q3.shape

(43,)

Our first model with ARIMA is a simple one, we'll allocate our capital totally to bitcoin if we predict that the next day price will rise, otherwise we will short 100%.

The total return for Q3 is 3.3%, not very impressive, but for Q4, it's 82%, which is much better. Return for 2 quarters is 87%

In [31]:
model = fit_arima(bitcoin_q2)
return_pred = model.predict(start=bitcoin_q2.shape[0], end=bitcoin_q2.shape[0] + bitcoin_q3.shape[0] - 1).values
signal = np.where(return_pred > 0, 1, -1)
return_q3 = np.product(bitcoin_q3 * signal + 1) - 1
return_q3

C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likeliho

0.03305217361367663

In [32]:
model = fit_arima(bitcoin_q3)
return_pred = model.predict(start=bitcoin_q3.shape[0], end=bitcoin_q3.shape[0] + bitcoin_q4.shape[0] - 1).values
signal = np.where(return_pred >0, 1, -1)
return_q4 = np.product(bitcoin_q4 * signal + 1) - 1
return_q4

C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at 

0.8192927891621655

In [33]:
(1 + return_q3) * (1 + return_q4) - 1

0.8794243702836635

We prefer the ARMA model for the ease of model and a decent return

# 6. Category 2 Models: Just use 2 variables

With VARMA model, we use the equity returns as to enhance the prediction for the bitcoin returns. for Q3, the return is 28% while for Q4, it's 63%.  Return for 2 quarters is 108%

In [34]:
model = fit_arima(bitcoin_q2, equity_q2)
return_pred = model.predict(exog=equity_q3, start=bitcoin_q2.shape[0], end=bitcoin_q2.shape[0] + bitcoin_q3.shape[0] - 1).values
signal = np.where(return_pred > 0, 1, -1)
return_q3 = np.product(bitcoin_q3 * signal + 1) - 1
return_q3

C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likeliho

0.2782697582897862

In [35]:
model = fit_arima(bitcoin_q3, equity_q3)
return_pred = model.predict(exog=equity_q4, start=bitcoin_q3.shape[0], end=bitcoin_q3.shape[0] + bitcoin_q4.shape[0] - 1).values
signal = np.where(return_pred > 0, 1, -1)
return_q4 = np.product(bitcoin_q4 * signal + 1) - 1
return_q4

C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at 

0.6289117237103818

In [36]:
(1 + return_q3) * (1 + return_q4) - 1

1.0821885953426689

We prefer the VARMA model thanks to its execellent return

# 7 . Category 3 Models: Use all 3 variables

## VARMA model

With VARMA model, we use both the equity and gold returns as to enhance the prediction for the bitcoin returns. for Q3, the return is 50% while for Q4, it's 40%. Return for 2 quarters is 109%

In [37]:
model = fit_arima(bitcoin_q2, pd.concat([equity_q2, gold_q2], axis=1))
return_pred = model.predict(exog=pd.concat([equity_q3, gold_q3], axis=1), start=bitcoin_q2.shape[0], end=bitcoin_q2.shape[0] + bitcoin_q3.shape[0] - 1).values
signal = np.where(return_pred > 0, 1, -1)
return_q3 = np.product(bitcoin_q3 * signal + 1) - 1
return_q3

C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likeliho

0.4981117995367046

In [38]:
model = fit_arima(bitcoin_q3, pd.concat([equity_q3, gold_q3], axis=1))
return_pred = model.predict(exog=pd.concat([equity_q4, gold_q4], axis=1), start=bitcoin_q3.shape[0], end=bitcoin_q3.shape[0] + bitcoin_q4.shape[0] - 1).values
signal = np.where(return_pred > 0, 1, -1)
return_q4 = np.product(bitcoin_q4 * signal + 1) - 1
return_q4

C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Python38\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likeliho

0.3971697263227958

In [39]:
(1 + return_q3) * (1 + return_q4) - 1

1.0931164529596487

In [40]:
# index = 2
# df = dfs[index]
# print(f"{model} model for {df_names[index]} from April to December")
# data = get_data(df, 4, 6, "Daily Return")
# data = data.dropna()
# garch_fitted = fit_garch(data, "GARCH")
# forecasts = garch_fitted.forecast(horizon=43)

Compare models using cumulative return and volatility

In [41]:
data1 = get_data(gold_df, 4, 6, "Daily Return").values
data2 = get_data(equity_df, 4, 6, "Daily Return").values
data3 = get_data(bitcoin_df, 4, 6, "Daily Return").values

In [42]:
coint(data1, np.array([data2, data3]).T)

(-6.076460637243138,
 5.692753720973776e-06,
 array([-4.69574274, -3.97301265, -3.618289  ]))

In [43]:
coint(data2, np.array([data1, data3]).T)

(-6.203463603398757,
 2.972096825666223e-06,
 array([-4.69574274, -3.97301265, -3.618289  ]))

In [44]:
coint(data3, np.array([data1, data2]).T)

(-5.93178147713331,
 1.1733043376457846e-05,
 array([-4.69574274, -3.97301265, -3.618289  ]))

p-value < 0.01 for all tests, we conclude that there are cointegrating vectors.

## Vector Error Correction Model

We allocate our capital to 3 assets based on the predicted return, with a negative return means that we'll short the asset, and the sum of the 3 assets is always 1. Q3's return is 23%, Q4's return is 47%, total return is 81%

In [45]:
data_q2 = np.array([data1, data2, data3]).T
model = VECM(data_q2, coint_rank=1)
vecm_res = model.fit()
res = vecm_res.predict(steps=43)
row_sums = res.sum(axis=1)
weights = res / row_sums[:, np.newaxis]
weights

array([[ 0.35615871, -0.13442721,  0.77826851],
       [ 0.76760722,  0.01941791,  0.21297487],
       [ 0.59149272, -0.09495761,  0.5034649 ],
       [ 0.67734894, -0.26858181,  0.59123287],
       [ 0.59466803, -0.17363254,  0.5789645 ],
       [ 0.62066518, -0.14095401,  0.52028883],
       [ 0.62382008, -0.1626765 ,  0.53885642],
       [ 0.62423843, -0.1768401 ,  0.55260166],
       [ 0.61861166, -0.16729426,  0.5486826 ],
       [ 0.62072114, -0.16345469,  0.54273354],
       [ 0.62163705, -0.16658122,  0.54494417],
       [ 0.62130375, -0.16791401,  0.54661026],
       [ 0.62080003, -0.16672788,  0.54592785],
       [ 0.62102733, -0.1663406 ,  0.54531327],
       [ 0.62114845, -0.16675154,  0.54560309],
       [ 0.62108596, -0.16687495,  0.54578899],
       [ 0.62103702, -0.16672607,  0.54568905],
       [ 0.62106491, -0.16669045,  0.54562555],
       [ 0.62107859, -0.16674252,  0.54566393],
       [ 0.62106946, -0.16675307,  0.54568362],
       [ 0.62106462, -0.16673461,  0.545

In [46]:
data1 = get_data(gold_df, 7, 9, "Daily Return").values
data2 = get_data(equity_df, 7, 9, "Daily Return").values
data3 = get_data(bitcoin_df, 7, 9, "Daily Return").values
data_q3 = np.array([data1, data2, data3]).T

In [47]:
return_q3 = weights * data_q3
total_return_q3 = return_q3.sum(axis=1)
portfolio_return_q3 = np.product(total_return_q3 + 1) - 1
portfolio_return_q3

0.23021595813304496

In [48]:
coint(data1, np.array([data2, data3]).T)

(-6.922723638048469,
 5.848143640071725e-08,
 array([-4.6556178 , -3.95031996, -3.60224723]))

In [49]:
coint(data2, np.array([data1, data3]).T)

(-3.7317779390504793,
 0.05119188151529683,
 array([-4.6556178 , -3.95031996, -3.60224723]))

In [50]:
coint(data3, np.array([data1, data2]).T)

(-8.835667550986745,
 3.3294143920743777e-13,
 array([-4.6556178 , -3.95031996, -3.60224723]))

Interestingly, we can't reject the null hypothesis for the test for the equity ETF and the combintion of gold and bitcoin ETF. We can reject the null hypothesis for the other two tests.

In [51]:

model = VECM(data_q3, coint_rank=1)
vecm_res = model.fit()
res = vecm_res.predict(steps=42)
row_sums = res.sum(axis=1)
weights = res / row_sums[:, np.newaxis]
weights

array([[ 0.36787808, -0.02095399,  0.65307591],
       [ 0.4516389 , -0.05938162,  0.60774272],
       [ 0.36473565, -0.09113523,  0.72639958],
       [ 0.44790039, -0.13803918,  0.69013879],
       [ 0.37440332, -0.08996644,  0.71556312],
       [ 0.42211176, -0.11344712,  0.69133536],
       [ 0.38806927, -0.09945866,  0.71138939],
       [ 0.41008717, -0.1083739 ,  0.69828674],
       [ 0.39550792, -0.10253513,  0.70702721],
       [ 0.40460034, -0.10597913,  0.70137879],
       [ 0.39894411, -0.10395701,  0.70501289],
       [ 0.40236036, -0.10512746,  0.7027671 ],
       [ 0.40032143, -0.10445588,  0.70413444],
       [ 0.40151098, -0.10482965,  0.70331867],
       [ 0.40082905, -0.10462671,  0.70379767],
       [ 0.40121154, -0.10473376,  0.70352222],
       [ 0.40100184, -0.10467928,  0.70367743],
       [ 0.40111364, -0.10470559,  0.70359195],
       [ 0.40105605, -0.10469385,  0.7036378 ],
       [ 0.40108438, -0.10469838,  0.70361401],
       [ 0.40107136, -0.10469719,  0.703

In [52]:
data1 = get_data(gold_df, 10, 12, "Daily Return").values
data2 = get_data(equity_df, 10, 12, "Daily Return").values
data3 = get_data(bitcoin_df, 10, 12, "Daily Return").values
data_q4 = np.array([data1, data2, data3]).T

In [53]:
return_q4 = weights * data_q4
total_return_q4 = return_q4.sum(axis=1)
portfolio_return_q4 = np.product(total_return_q4 + 1) - 1
portfolio_return_q4

0.47305473104700324

In [55]:
(1 + portfolio_return_q3) * (1 + portfolio_return_q4) - 1

0.8121754373374039

We prefer the VARMA model because it has a better return

# 8. Combining the models

Our 2 VARMA models have the best returns, we will naturally choose these two models to combine. Because the returns are close, we assign each model 50% of our capital. We will not rebalance the portfolio during the whole periods.

The return of this model, therefore, will be the weighted average of the 2 models, which is about 108.7%

In a sophisticated trading system, we can also use Modern portfolio theory to allocate the assets in a portfolio that maximizes the expected return.